The goal of this notebook is to try and accumulate all of the previous information gathered regarding the instagram engagements with other information, and see if there is a way to accumulate it into one space in order to make this into a deliverable product. 

This means the joining of the actual instagram dataset, and then the extra research performed on the hashtags and @'s attached in each description.

# Step 0 : Import dependencies and visualization tools

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('coolwarm')
%matplotlib inline

# Step 1 : Load in Instagram Dataset, Derive Preliminary Features that require no outside data. 

In [2]:
instas = pd.read_csv('Business Analytics/training_set.csv',encoding = 'unicode_escape')
instas.fillna('',inplace=True)

In [11]:
instas['Description_Len'] = instas['Description'].apply(len)
instas['num@s'] = instas['Description'].apply(lambda z: z.count('@'))
instas['num#s'] = instas['Description'].apply(lambda z: z.count('#'))
instas['num?s'] = instas['Description'].apply(lambda z: z.count('?')) # a shitty proxy for emojis
import re
instas['numCAPs'] = instas['Description'].apply(lambda z: len(re.findall(r'[A-Z]',z)))

def find_dunks(z):
    dunk_words = ['slam','jam','finish','dunk','putback','rim','windmill','flush','oop']
    for dw in dunk_words:
        if dw in z.lower(): 
            return 1
    return 0

def find_buzzer_beaters(z):
    buzzer_beater_words = ['buzz','beat','clock']
    
    for bbw in buzzer_beater_words:
        if bbw in z.lower():
            return 1
    return 0

instas['Buzzer Beater?'] = instas['Description'].apply(lambda z: find_buzzer_beaters(z)) #shitty proxy for buzzer beaters
instas['Dunk?'] = instas['Description'].apply(lambda z:  find_dunks(z)) #shitty proxy for buzzer beaters


import datetime

instas['Timezone'] = instas['Created'].str.split(' ',expand =True)[2]
instas['Date'] = instas['Created'].str.split(' ',expand = True)[0]
instas['Month'] = instas['Date'].str.split('-',expand = True)[1]

instas['Time'] = instas['Created'].str.split(' ',expand = True)[1]
instas['Hour'] = instas['Time'].str.split(':',expand=True)[0]

instas['Date'] = pd.to_datetime(instas['Date']).astype(datetime.datetime)

instas['Weekday'] = instas['Date'].apply(lambda z: z.weekday())
instas['Year'] = instas['Created'].apply(lambda z: z.split('-')[0])

In [12]:
instas.drop(columns=['Date','Time'],inplace=True)

# Step 2 Load In Exogeneous Features Such as Hashtag Popularity and Profile Clout

In [ ]:
# Please do this here soon!

In [13]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

X = instas.drop(columns=['Engagements','Description','Created','Followers at Posting']).copy()

X = pd.get_dummies(X)

X_norm  = normalize(X)

y = instas['Engagements']

In [20]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.4)

model = xgb.XGBRegressor(n_estimators=550,booster='dart')#choice(models)

print("Model used: ", model)
model.fit(X_train,y_train)
print("Training score: ", model.score(X_train,y_train))
print("Original Score", model.score(X_test,y_test))

def mean_absolute_percentage_error(y_true, y_pred):
    import numpy as np
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true = y_test
y_pred = model.predict(X_test)

#y_pred = np.array([round(y) for y in y_pred])

print("MAPE Original Score ", mean_absolute_percentage_error(y_true, y_pred))


print("Jordan's MAPE, ",  MAPE_np(y_true,y_pred))

Model used:  XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=550,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Training score:  0.9315375993602187
Original Score 0.9019930399080099
MAPE Original Score  6.6013107333303855
Jordan's MAPE,  6.6013107333303855
